In [1]:
import pandas as pd
from scipy.spatial import distance
import zipfile
import os.path
import os
from scipy.stats import rankdata
import operator
from numpy import dot
from numpy.linalg import norm

In [3]:
df_book = pd.read_csv("../book-vec-group.csv")
df_book

,Unnamed: 0,Book-Title,Book-Author_x,Book-Summary,ISBN,group,BERT,word2vec
0,0,A Clockwork Orange,Anthony Burgess,"Alex, a teenager living in near-future Englan...",345316413X,23,"[-0.29636892676353455, 0.8470680117607117, 0.7...",[ 4.51489369e-02 6.01137093e-02 -1.26928838e-...
1,1,The Plague,Albert Camus,The text of The Plague is divided into five p...,0075536498,47,"[0.2956414222717285, 1.319756031036377, 0.3554...",[ 0.05818041 0.06845398 0.00783395 0.036492...
2,2,An Enquiry Concerning Human Understanding,David Hume,The argument of the Enquiry proceeds by a ser...,0872202291,5,"[1.2887966632843018, 1.2676935195922852, 1.117...",[ 0.06308864 0.02407891 0.03377491 0.085396...
3,3,All Quiet on the Western Front,Erich Maria Remarque,"The book tells the story of Paul Bäumer, a Ge...",0449213943,8,"[0.2628381550312042, 1.452850103378296, 1.0604...",[ 0.08981435 0.08089734 0.02092221 0.061505...
4,4,A Wizard of Earthsea,Ursula K. Le Guin,"Ged is a young boy on Gont, one of the larger...",0689317204,32,"[-0.12000640481710434, 1.7298859357833862, 0.7...",[ 0.07931867 0.07550479 -0.00114273 0.017464...
...,...,...,...,...,...,...,...,...
5845,5845,Gates of Paradise,V. C. Andrews,"A novel about Annie Stonewall, the daughter o...",0671670646,14,"[0.37652456760406494, 1.5239266157150269, 0.91...",[ 0.02671554 -0.01298588 -0.03296045 0.033805...
5846,5846,Shelter,Harlan Coben,"After Mickey Bolitar moves in with his uncle,...",0385313896,11,"[-0.22981029748916626, 1.2231945991516113, 1.8...",[ 0.07016824 0.04757968 -0.036401 0.025883...
5847,5847,Remote Control,Andy McNab,The series follows the character of Nick Ston...,0345428056,43,"[0.23090682923793793, 0.9125347137451172, 0.59...",[ 0.04248047 0.07196384 -0.0039071 0.050720...
5848,5848,The Simpsons: A Complete Guide to Our Favorite...,Matt Groening,"{| class=""wikitable"" |- !Seasons covered !Boo...",0060952520,3,"[0.0948098748922348, 1.4162523746490479, 0.936...",[-2.14607932e-02 -4.20726429e-02 -3.74117765e-...


In [9]:
def recommend_on_group(book_title):
    group = df_book[df_book["Book-Title"] == book_title]["group"].tolist()[0]
    recommend_list = df_book[df_book["group"] == group]["Book-Title"].tolist()
    return recommend_list

recommend_on_group('Dune')

['Julius Caesar',
 'The Crow Road',
 'Titus Andronicus',
 'Excession',
 'Dune',
 'Sick Puppy',
 'Tartarin de Tarascon',
 'The Robe',
 'Murder in the Cathedral',
 'The Children of Men',
 "The Pilgrim's Progress",
 'Shogun: A Novel of Japan',
 'Roadmarks',
 'The Chrysalids',
 'Pyramids',
 'Burmese Days',
 'The Golden Age',
 'Lucky Jim',
 "A Connecticut Yankee in King Arthur's Court",
 'Billy Budd',
 'Metropolitan',
 'Men at Arms',
 'Henry VIII',
 'Looking Backward',
 'The Moon Is Down',
 'Journey to the Center of the Earth',
 'The Leopard',
 'The Cruel Sea',
 'The Gemini Contenders',
 'The Smoke Ring',
 'Asterix in Britain',
 'Whirlwind',
 'To Your Scattered Bodies Go',
 'The Dark Design',
 'Orange Crush',
 'The First Man in Rome',
 'The Canterville Ghost',
 'The Peshawar Lancers',
 'The Thirteen-Gun Salute',
 'Asterix the Gladiator',
 'The Eagle of the Ninth',
 'The Lantern Bearers',
 'Waiting for the Barbarians',
 'The Brothers Lionheart',
 'Infernal Devices',
 "Hannibal's Children",
 

In [33]:
def strtoVecBERT(str):
    str = str[1:len(str) - 1]
    list_str = list(str.split(","))
    list_float = []
    for element in list_str:
        list_float.append(float(element))
    return list_float

def strtoVecword2vec(str):
    str = str[1:len(str) - 1]
    list_str = list(str.split(" "))
    list_float = []
    for element in list_str:
        if len(element) > 0:
            list_float.append(float(element))
    return list_float
 
# Driver code
str1 = df_book[df_book["Book-Title"] == "Firewall"]["word2vec"].tolist()[0]
str2 = df_book[df_book["Book-Title"] == "Misspent Youth"]["word2vec"].tolist()[0]
vec1 = strtoVecword2vec(str1)
vec2 = strtoVecword2vec(str2)
distance_self = cosine_distance(vec1, vec1)
distance_self

2.220446049250313e-16

In [43]:
def recommend_one_on_BERT(book_title):
    vec1 = strtoVecBERT(df_book[df_book["Book-Title"] == book_title]["BERT"].tolist()[0])
    min_distance = 10000
    most_similar_book = ''
    for index, row in df_book.iterrows():
        vec2 = strtoVecBERT(row["BERT"])
        dist = cosine_distance(vec1, vec2) 
        if(dist < min_distance and dist >= 1e-15):
            min_distance = dist
            most_similar_book = row["Book-Title"]

    return most_similar_book

recommend_one_on_BERT("Firewall")

'Sidetracked'

In [39]:
def recommend_one_on_word2vec(book_title):
    vec1 = strtoVecword2vec(df_book[df_book["Book-Title"] == book_title]["word2vec"].tolist()[0])
    min_distance = 10000
    most_similar_book = ''
    for index, row in df_book.iterrows():
        vec2 = strtoVecword2vec(row["word2vec"])
        dist = cosine_distance(vec1, vec2) 
        if(dist < min_distance and dist >= 1e-15):
            min_distance = dist
            most_similar_book = row["Book-Title"]

    return most_similar_book

recommend_one_on_word2vec("Firewall")

'The Man Who Never Missed'

In [48]:
def cosine_distance(vec1, vec2):
    cos_distance = 1 - dot(vec1, vec2) / (norm(vec1) * norm(vec2))
    return cos_distance

def recommend_n_on_ensemble(book_title, n):
    word2vec_list = []
    vec1 = strtoVecword2vec(df_book[df_book["Book-Title"] == book_title]["word2vec"].tolist()[0])
    for index, row in df_book.iterrows():
        vec2 = strtoVecword2vec(row["word2vec"])
        dist = cosine_distance(vec1, vec2) 
        title2 = row["Book-Title"]
        word2vec_list.append((title2, dist))
    
    word2vec_sorted = sorted(word2vec_list, key=lambda tup: tup[1])[0:len(word2vec_list)]
    word2vec_dist = [x[1] for x in word2vec_sorted]
    word2vec_rank = rankdata(word2vec_dist)

    bert_list = []
    vec1 = strtoVecBERT(df_book[df_book["Book-Title"] == book_title]["BERT"].tolist()[0])
    for index, row in df_book.iterrows():
        vec2 = strtoVecBERT(row["BERT"])
        dist = cosine_distance(vec1, vec2) 
        title2 = row["Book-Title"]
        bert_list.append((title2, dist))

    bert_sorted = sorted(bert_list, key=lambda tup: tup[1])[0:len(bert_list)]
    bert_dist = [x[1] for x in bert_sorted]
    bert_name = [x[0] for x in bert_sorted]
    bert_rank = rankdata(bert_dist)

    scores = []
    for i in range(len(word2vec_rank)):
       score = 1/ (i + 1)  #rank of word2vec; add 1 to avoid division by 0
       index_bert_name = bert_name.index(word2vec_sorted[i][0])
       rank = bert_rank[index_bert_name]
       score = score + 1 / (rank + 1) 
       scores.append((word2vec_sorted[i][0], score))
    
    scores_sorted = sorted(scores, key=lambda tup: tup[1])[:n]
    book_list = [x[0] for x in scores_sorted]
    return book_list

book_list = recommend_n_on_ensemble("Term Limits", 10)
book_list

['Ceremony',
 'Jackals',
 "There's a Girl in My Hammerlock",
 'The Ugly Duckling',
 'How to Eat Fried Worms',
 'Mouse Soup',
 'The Affair',
 'White Snow, Bright Snow',
 'Here Be Monsters',
 'Tommy Trouble and the Magic Marble']

In [59]:
def recommend_on_list(book_list, n):
    recommend_list = []
    #print(book_list)
    for book in book_list:
        recommend_books = recommend_n_on_ensemble(book, 10)
        for recommend_book in recommend_books:
            if recommend_book not in recommend_list:
                recommend_list.append(recommend_book)
    return recommend_list[:n]



book_list = ['Ceremony','Jackals',"There's a Girl in My Hammerlock",
 'The Ugly Duckling',
 'How to Eat Fried Worms',
 'Mouse Soup',
 'The Affair',
 'White Snow, Bright Snow',
 'Here Be Monsters',
 'Tommy Trouble and the Magic Marble']

recommend_on_list(book_list, 10)

['The Eyes of Darkness',
 'Flame',
 'Butterfly',
 'The Saint and the Hapsburg Necklace',
 'Grania: She-King of the Irish Seas',
 'Irish Linen',
 'Term Limits',
 'Irish Stew!',
 'Archform: Beauty',
 'The Apprentice']

In [54]:
def isbn_to_title(isbn):
    row = df_book[df_book["ISBN"] == isbn]
    return row["Book-Title"].to_list()[0]

isbn_to_title('0075536498')

'The Plague'

In [30]:
book_list1 = ['A Clockwork Orange', 'Gates of Paradise']
book_list2 = ['A Clockwork Orange', 'Shelter']
def compare_on_list(book_list1, book_list2):
    group_list1 = []
    group_list2 = []
    for book in book_list1:
        row = df_book[df_book["Book-Title"] == book]
        group = row["group"].to_list()[0]
        group_list1.append(group)
    
    for book in book_list2:
        row = df_book[df_book["Book-Title"] == book]
        group = row["group"].to_list()[0]
        group_list2.append(group)
    
    common_groups_list = []
    common_groups = 0
    i = 0
    for group1 in group_list1:
        j = 0
        for group2 in group_list2:
            if group1 == group2:
                common_groups += 1
                common_groups_list.append((book_list1[i], book_list2[j]))    
            j += 1
        i += 1

    return common_groups, common_groups_list

compare_on_list(book_list1, book_list2)   

(1, [('A Clockwork Orange', 'A Clockwork Orange')])